In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import chi2

#input_data
PATH = r"C:\Users\bhara\Documents\Airlink project\MLRData.xlsx"
df = pd.read_excel(PATH)

TARGET = "Aircraft_Type"

PREDICTORS = [
    "AW",
    "Cost",
    "Distance",
]

#needed_columns
needed = [TARGET] + PREDICTORS
missing_cols = [c for c in needed if c not in df.columns]
if missing_cols:
    raise ValueError(
        f"These columns are not in the file: {missing_cols}\n"
        f"Available columns: {df.columns.tolist()}"
    )

data = df[needed].copy().reset_index(drop=True)

#clean_target
data[TARGET] = data[TARGET].astype("object")
data = data[data[TARGET].notna()]
data = data[data[TARGET].astype(str).str.strip() != ""]
data = data.reset_index(drop=True)

#clean_predictors
for col in PREDICTORS:
    data[col] = pd.to_numeric(data[col], errors="coerce")
    med = data[col].median()
    if np.isnan(med):
        med = 0.0
    data[col] = data[col].fillna(med)

#encoding_target_variables
data["y_code"] = data[TARGET].astype("category").cat.codes
classes = list(data[TARGET].astype("category").cat.categories)

print("Classes (code -> label):", dict(enumerate(classes)))
print("N rows used:", len(data))

#formula_builder
def build_formula(predictors):
    if len(predictors) == 0:
        return "y_code ~ 1"
    rhs = " + ".join(predictors)
    return f"y_code ~ {rhs}"

#MNLR_and_AIC_BIC
rows = []
prev_res = None

for k in range(0, len(PREDICTORS) + 1):
    current_preds = PREDICTORS[:k]
    formula = build_formula(current_preds)

    res = smf.mnlogit(
        formula,
        data=data
    ).fit(
        method="newton",
        maxiter=200,
        disp=False
    )

    llf = res.llf
    n = int(res.nobs)
    k_params = int(res.params.size)

    aic = -2 * llf + 2 * k_params
    bic = -2 * llf + np.log(n) * k_params

    row = {
        "step": k,
        "added": None if k == 0 else PREDICTORS[k-1],
        "logLik": float(llf),
        "AIC": float(aic),
        "BIC": float(bic),
        "k_params": k_params,
        "LR_stat_vs_prev": np.nan,
        "LR_df": np.nan,
        "LR_pvalue": np.nan,
    }

    if prev_res is not None:
        LR = 2 * (res.llf - prev_res.llf)
        df_diff = int(res.params.size - prev_res.params.size)
        pval = chi2.sf(LR, df_diff) if df_diff > 0 else np.nan

        row.update({
            "LR_stat_vs_prev": float(LR),
            "LR_df": float(df_diff),
            "LR_pvalue": float(pval),
        })

    rows.append(row)
    prev_res = res

results = pd.DataFrame(rows)
results["delta_logLik"] = results["logLik"].diff()
results["delta_AIC"] = results["AIC"].diff()
results["delta_BIC"] = results["BIC"].diff()

print("\n=== Stepwise Multinomial Logit Results ===")
print(results[[
    "step",
    "added",
    "logLik",
    "delta_logLik",
    "AIC",
    "delta_AIC",
    "BIC",
    "delta_BIC",
    "LR_stat_vs_prev",
    "LR_df",
    "LR_pvalue"
]].to_string(index=False))

#final_model
final_formula = build_formula(PREDICTORS)
final_res = smf.mnlogit(
    final_formula,
    data=data
).fit(
    method="newton",
    maxiter=200,
    disp=False
)

print("Final Model Summary")
print(final_res.summary())

#pairwise_comparisons_all_classes
import re
from scipy.stats import norm, chi2

params = final_res.params
cov = final_res.cov_params()

J = len(classes)
exog_names = list(params.index)
k_exog = len(exog_names)

#detect_base_class
def _col_to_int(c):
    if isinstance(c, (int, np.integer)):
        return int(c)
    s = str(c)
    m = re.search(r"(\d+)$", s)
    return int(m.group(1)) if m else None

col_codes = [_col_to_int(c) for c in params.columns]
all_codes = set(range(J))
nonbase_codes = set(col_codes)
base_code = list(all_codes - nonbase_codes)[0]
base_label = classes[base_code]

print(f"Base class: {base_label}")

code_to_label = {i: classes[i] for i in range(J)}
code_to_colpos = {code: j for j, code in enumerate(col_codes)}

#helpers
def get_beta_and_cov(code):
    if code == base_code:
        beta = np.zeros(k_exog)
        V = np.zeros((k_exog, k_exog))
        return beta, V

    j = code_to_colpos[code]
    beta = params.iloc[:, j].to_numpy()
    sl = slice(j * k_exog, (j + 1) * k_exog)
    V = cov.iloc[sl, sl].to_numpy()
    return beta, V

def get_cross_cov(code_a, code_b):
    if code_a == base_code or code_b == base_code:
        return np.zeros((k_exog, k_exog))

    ja = code_to_colpos[code_a]
    jb = code_to_colpos[code_b]
    sla = slice(ja * k_exog, (ja + 1) * k_exog)
    slb = slice(jb * k_exog, (jb + 1) * k_exog)
    return cov.iloc[sla, slb].to_numpy()

#pairwise_wald_tests
pairwise_rows = []
joint_rows = []

for a in range(J):
    for b in range(a + 1, J):
        beta_a, V_a = get_beta_and_cov(a)
        beta_b, V_b = get_beta_and_cov(b)
        C_ab = get_cross_cov(a, b)

        d = beta_a - beta_b
        Vd = V_a + V_b - C_ab - C_ab.T

        se = np.sqrt(np.clip(np.diag(Vd), 1e-15, None))
        z = d / se
        p = 2 * (1 - norm.cdf(np.abs(z)))
        OR = np.exp(d)

        for i, name in enumerate(exog_names):
            pairwise_rows.append({
                "A": code_to_label[a],
                "B": code_to_label[b],
                "Predictor": name,
                "log_odds_diff(A_vs_B)": d[i],
                "SE": se[i],
                "z": z[i],
                "p": p[i],
                "odds_ratio(A_vs_B)": OR[i],
            })

        idx = [i for i, n in enumerate(exog_names) if n.lower() != "intercept"]
        d_sub = d[idx]
        Vd_sub = Vd[np.ix_(idx, idx)]

        try:
            W = float(d_sub.T @ np.linalg.inv(Vd_sub) @ d_sub)
            df_w = len(d_sub)
            p_w = float(chi2.sf(W, df_w))
        except np.linalg.LinAlgError:
            W, df_w, p_w = np.nan, len(d_sub), np.nan

        joint_rows.append({
            "A": code_to_label[a],
            "B": code_to_label[b],
            "Wald_chi2(all_predictors)": W,
            "df": df_w,
            "p": p_w,
        })

pairwise_df = pd.DataFrame(pairwise_rows)
joint_df = pd.DataFrame(joint_rows)

print("Pairwise Predictor-Level Comparisons (A vs B)")
print(pairwise_df.to_string(index=False))

#print("\n=== Pairwise Joint Wald Tests (A vs B) ===")
#print(joint_df.to_string(index=False))

Classes (code -> label): {0: 'Freighter', 1: 'Narrowbody', 2: 'Widebody'}
N rows used: 539

=== Stepwise Multinomial Logit Results ===
 step    added      logLik  delta_logLik        AIC   delta_AIC        BIC  delta_BIC  LR_stat_vs_prev  LR_df    LR_pvalue
    0     None -359.814404           NaN 723.628809         NaN 732.208240        NaN              NaN    NaN          NaN
    1       AW -339.633935     20.180469 687.267870  -36.360938 704.426732 -27.781507        40.360938    2.0 1.720813e-09
    2     Cost -338.942154      0.691781 689.884308    2.616438 715.622601  11.195869         1.383562    2.0 5.006835e-01
    3 Distance -285.699878     53.242276 587.399755 -102.484553 621.717480 -93.905121       106.484553    2.0 7.536561e-24
Final Model Summary
                          MNLogit Regression Results                          
Dep. Variable:                 y_code   No. Observations:                  539
Model:                        MNLogit   Df Residuals:                   